In [44]:
# initialize project:
using Pkg; Pkg.activate("./MyProject");;
include("libs/local_clifford_group.jl"); include("libs/cnc.jl"); include("libs/utils.jl");

using GAP
const g = GAP.Globals;

Generate Clifford group and local Clifford group

In [46]:
# 3-qubit local Clifford group:
LC2 = LocalCliffordGroup(2); local_cl2 = LC2.LocalCliffGroup; PS2 = PauliString(2);

In [6]:
# 3-qubit local Clifford group:
LC3 = LocalCliffordGroup(3); local_cl3 = LC3.LocalCliffGroup; PS3 = PauliString(3);

In [7]:
# 3-qubit Clifford group
C3 = CliffordGroup(3); cl3 = C3.CliffGroup;

Generate LC phase space:

In [8]:
d = Vector{Int}([1 for i in 1:64]);
D = array_to_matrix(local_clifford_orbit_of_point(LC3,d)); 

In [9]:
cncs = generate_all_cncs(3,[1,2,3]);
CNC = generate_cnc_vertices(3,cncs);

In [10]:
V = Matrix{Rational{Int64}}(hcat(D,CNC))

64×71648 Matrix{Rational{Int64}}:
 1  1   1   1  1   1   1   1   1   1  …   1   1   1   1   1  1   1   1   1
 1  1   1   1  1   1   1   1   1  -1      0   0   0   1   1  0   0   0   1
 1  1   1  -1  1   1  -1   1  -1  -1      0   0   0  -1   0  0   0   0   0
 1  1   1   1  1   1   1   1   1   1      0   0   0   0   0  0  -1   0   0
 1  1   1   1  1   1   1  -1   1   1      0   0   0   0   0  0   0   0   0
 1  1   1   1  1   1   1  -1   1  -1  …   0   0   0   0   0  1   0   1   0
 1  1   1  -1  1   1  -1  -1  -1  -1      0   0   0   0  -1  0   0   0   0
 1  1   1   1  1   1   1  -1   1   1      0  -1   0  -1   0  0   0  -1  -1
 1  1  -1   1  1  -1   1  -1  -1   1      0  -1   1   0   0  0   0   1   0
 1  1  -1   1  1  -1   1  -1  -1  -1      1   0   0   0   0  0   0   0   0
 ⋮                 ⋮                  ⋱       ⋮                  ⋮      
 1  1   1   1  1   1   1  -1   1   1  …   0   0   0   0   1  0   0   0   0
 1  1  -1   1  1  -1   1  -1  -1   1      0   0   0   0   0  1   0  

In [11]:
# Define magic state
T = Vector{Real}([1,1/sqrt(2),1/sqrt(2),0]);
T3 = Vector{Float64}(kron(kron(T,T),T));;

In [12]:
# Define controlled Z gate:
for g in C3.NormalizerGates.Gens; println(g); end

GAP: [ [ 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0 ], [ 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0 ], [ 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0 ], [ 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 1/2*E(8)-1/2*E(8)^3 ], [ 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, -1/2*E(8)+1/2*E(8)^3, 0, 0, 0 ], [ 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, -1/2*E(8)+1/2*E(8)^3, 0, 0 ], [ 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, -1/2*E(8)+1/2*E(8)^3, 0 ], [ 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, -1/2*E(8)+1/2*E(8)^3 ] ]
GAP: [ [ 1, 0, 0, 0, 0, 0, 0, 0 ], [ 0, 1, 0, 0, 0, 0, 0, 0 ], [ 0, 0, 1, 0, 0, 0, 0, 0 ], [ 0, 0, 0, 1, 0, 0, 0, 0 ], [ 0, 0, 0, 0, E(4), 0, 0, 0 ], [ 0, 0, 0, 0, 0, E(4), 0, 0 ], [ 0, 0, 0, 0, 0, 0, E(4), 0 ], [ 0, 0, 0, 0, 0, 0, 0, E(4) ] ]
GAP: [ [ 1/2*E(8)-1/2*E(8)^3, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 0, 0 ], [ 0, 1/2*E(8)-1/2*E(8)^3, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 0 ], [ 1/2*E(8)-1/2*E(8)^3, 0, -1/2*E(8)+1/2*E(8)^3, 0, 0, 0, 0, 0 ], [ 0, 1/2*E(8)-1/2*E(8)^3, 0, -1/

In [13]:
# Unitaries:
CX12 = cnot(1,2,3); CX13 = cnot(1,3,3); CX23 = cnot(2,3,3);
H1 = hadamard_j(1,3); H2 = hadamard_j(2,3); H3 = hadamard_j(3,3);

In [14]:
cx12 = C3.GateGroupHom(CX12); cx13 = C3.GateGroupHom(CX13); cx23 = C3.GateGroupHom(CX23);
h1 = C3.GateGroupHom(H1); h2 = C3.GateGroupHom(H1); h3 = C3.GateGroupHom(H1); 
cz12 = h2*cx12*g.Inverse(h2); cz13 = h3*cx13*g.Inverse(h3); cz23 = h3*cx23*g.Inverse(h3);

In [15]:
function clifford_gate_action_by_gens(gens::Vector{GapObj},CG::CliffordGroup,C::Vector)
    C = Vector{Real}(C)
    fdict = CG.IntActionDict
    bdict = CG.ActionIntDict
    G = CG.CliffGroup.Grp
    N = length(C);
    K = g.Set(jlg([]))

    gg = gens[1];
    for elem in gens[2:end]; gg = elem*gg; end;

    vals, V, map = grouping(C)

    K = Vector{Vector{Int}}([])
    for v in V
        k = Vector{Int}([]);
        for e in v
            x = e; y = fdict[e]; gy = g.OnPoints(jlg(y),gg); gx = bdict[gy];
            #println("$x, $y, $gy, $gx")
            push!(k,gx); 
        end
        push!(K,k)
    end

    return subset_to_vector(N,K,map)
end

clifford_gate_action_by_gens (generic function with 1 method)

In [16]:
L3 = clifford_gate_action_by_gens([cz12,cz23],C3,T3);
K3 = clifford_gate_action_by_gens([cz12,cz23,cz13],C3,T3);

In [17]:
using JuMP

using JuMP
using GLPK
using LinearAlgebra

# Dimensions
d = size(V)[1]; N = size(V)[2];

In [18]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, V * x .== L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.0403731957466684
Optimal value of x: [0.027283033212800553, 0.013220698412909912, 0.0, 0.055036303621252336, 0.0, 0.0, 0.0021342708223135026, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006515782168014448, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03483125881400709, 0.0, 0.0, 0.015169225368281022, 0.0, 0.0, 0.0, 0.007506401367948554, 0.0, 0.061393153702887386, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006088338504120205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06419841804048952, 0.006047306484249535, 0.0, 0.0, 0.07427247782985812, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00887770963308433, -0.00021669379955314893, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0048223387642379115, -0.0031992928861683677, 0.0

In [19]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, V * x .== K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.0522970773009195
Optimal value of x: [0.0757910951603478, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0001734606680940697, -0.0055351130197758775, 0.0, 0.0, 0.009022817586849812, 0.0, 0.0, 0.01795890483965239, 0.0, 0.020179773960448524, 0.0, 0.011243686707645681, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.023754208861569398, -0.00553511301977576, 0.0, 0.0, 0.0, 0.0, 0.028942400545156826, 0.0, 0.0, 0.05382410374933855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005535113019775643, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.035171356237309534, 0.0, 0.0, 0.0, 0.0034877045670739216, 0.0, 0.0, 0.021793530742914895, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03883252147247772, 0.009283008588990919, 0.0, 0.0, 0.021966991411008656, 0.0, 0.010983495705504435, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005795304021917178, 0.0, 0.0, 0

Excessive output truncated after 524291 bytes.

0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [21]:
using JuMP
using GLPK
using LinearAlgebra

# Dimensions
d = size(V)[1]; N = size(CNC)[2];

Check that we recover correct value for CNC polytope.

In [22]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, CNC * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [23]:
obj_value

1.2828427124746187

LC phase space with only deterministic vertices 

In [28]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [29]:
obj_value

1.2437184335382283

In [30]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [31]:
obj_value

1.2071067811865475

LC phase space with deterministic vertices, max CNC operators, and tensored $2$-qubit vertices

In [54]:
using Polymake
const pm = Polymake;

include("libs/symmetries.jl");

Z = Vector{Rational{Int64}}([1,0,0,1])
ZZ = kron(Z,Z);
ZZZ = kron(Z,ZZ);

# Local 
S2 = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC2,ZZ)));
S3 = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC3,ZZZ)));

In [52]:
LL2 = pm.polytope.Polytope(INEQUALITIES = transpose(S2))

type: Polytope<Rational>

INEQUALITIES
  1   0   0   1  0  0  0  0   0   0   0  0   1   0   0   1
  1   0   0  -1  0  0  0  0   0   0   0  0   1   0   0  -1
  1   0   0   1  0  0  0  0   0   0   0  0  -1   0   0  -1
  1   0   0  -1  0  0  0  0   0   0   0  0  -1   0   0   1
  1  -1   0   0  0  0  0  0   0   0   0  0   1  -1   0   0
  1   1   0   0  0  0  0  0   0   0   0  0   1   1   0   0
  1  -1   0   0  0  0  0  0   0   0   0  0  -1   1   0   0
  1   1   0   0  0  0  0  0   0   0   0  0  -1  -1   0   0
  1   0   1   0  0  0  0  0   0   0   0  0   1   0   1   0
  1   0  -1   0  0  0  0  0   0   0   0  0   1   0  -1   0
  ⋮                 ⋮                 ⋮                  ⋮
  1  -1   0   0  0  0  0  0   1  -1   0  0   0   0   0   0
  1   1   0   0  0  0  0  0   1   1   0  0   0   0   0   0
  1  -1   0   0  0  0  0  0  -1   1   0  0   0   0   0   0
  1   1   0   0  0  0  0  0  -1  -1   0  0   0   0   0   0
  1   0   1   0  0  0  0  0   1   0   1  0   0   0   0   0
  1   0  -1   0  0  0  0  0   1   0  -1  0   0   0   0   0
  1   0   1   0  0  0  0  0  -1   0  -1  0   0   0   0   0
  1   0  -1   0  0  0  0  0  -1   0   1  0   0   0   0   0
  1   0   0   0  0  0  0  0   0   0   0  0   0   0   0   0

In [59]:
R2 = transpose(Matrix{Rational{Int64}}(representative_vertices(LL2)));
tR2 = kron(R2,Vector{Rational{Int64}}([1,1,1,1]));

In [66]:
R3 = Matrix{Rational{Int64}}(undef,64,0)
for k in 1:size(tR2)[2]
    Vk = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC3,tR2[:,k])))
    println(size(Vk))
    R3 = hcat(R3,Vk)
end

In [68]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [69]:
obj_value

0.9999999999999997

In [70]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [71]:
obj_value

1.0425339700071377

# $4$-qubit LC robustness

In [72]:
# 3-qubit local Clifford group:
LC4 = LocalCliffordGroup(34); local_cl4 = LC4.LocalCliffGroup; PS4 = PauliString(4);

In [ ]:
tR4 = kron(R2,Vector{Rational{Int64}}([1 for i in 1:16]));

In [ ]:
R4 = Matrix{Rational{Int64}}(undef,4^4,0)
for k in 1:size(tR4)[2]
    Vk = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC4,tR4[:,k])))
    println(size(Vk))
    R4 = hcat(R4,Vk)
end